In [14]:
import pandas as pd

action_1_path = "./data/JData_Action_201602.csv"
action_2_path = "./data/JData_Action_201603.csv"
action_2_extra_path = "./data/JData_Action_201603_extra.csv"
action_3_path = "./data/JData_Action_201604.csv"
comment_path = "./data/JData_Comment(修正版).csv"
product_path = "./data/JData_Product.csv"
user_path = "./data/JData_User.csv"

action_1 = pd.read_csv(action_1_path)
action_2 = pd.read_csv(action_2_path)
action_3 = pd.read_csv(action_3_path)
action_2_extra = pd.read_csv(action_2_extra_path)
comment = pd.read_csv(comment_path)
product = pd.read_csv(product_path)
user = pd.read_csv(user_path, encoding='gbk')

In [ ]:
def gen_user_feat():
    

In [15]:
# 用户数量 103616 

pd.get_dummies(user["age"])
age_df = pd.get_dummies(user["age"], prefix="age", columns=["0", "1", "2", "3", "4", "5", "6"])
sex_df = pd.get_dummies(user["sex"], prefix="sex")
user_lv_df = pd.get_dummies(user["user_lv_cd"], prefix="user_lv_cd")
user = pd.concat([user, age_df, sex_df ,user_lv_df], axis=1)
user.head()
#print user.head()



,user_id,age,sex,user_lv_cd,user_reg_dt,age_-1,age_15岁以下,age_16-25岁,age_26-35岁,age_36-45岁,age_46-55岁,age_56岁以上,sex_0,sex_1,sex_2,user_lv_cd_1,user_lv_cd_2,user_lv_cd_3,user_lv_cd_4,user_lv_cd_5
0,54,-1,2,1,2016/1/26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,79,36-45岁,2,2,2016/1/27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,100,16-25岁,2,3,2016/1/27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,266,15岁以下,2,2,2016/1/29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,333,-1,2,4,2016/1/30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [16]:
# 商品数量 24187 
# 类目数量 8
# 品牌数量 545
print product

       sku_id  attr1  attr2  attr3  cate  brand
0      166731      1     -1     -1     8    545
1      146488      2      2     -1     8    812
2       70546      3      1      1     8    214
3       92487      2      2     -1     8    812
4      113065      2      2     -1     8    812
5       80307      2      2     -1     8    812
6       71669      2      1     -1     8    812
7        4145      2      2      1     8    812
8       35450      2      2     -1     8    812
9      131403      2      1     -1     8    812
10     128621      2      1     -1     8    812
11      65347      2      1      2     8    812
12      68153      2      1      2     8    812
13      77945      2      1     -1     8    812
14     167028      2      1     -1     8    812
15      14771      2      1      2     8    812
16     153408      2      1     -1     8    812
17      39874      2      2     -1     8    812
18      83454      2      1     -1     8    812
19     141684      2     -1      2     8

In [17]:
product.brand.unique()

array([545, 812, 214, 623, 124, 658, 885, 403, 321, 306, 766, 800, 489,
       209, 244, 484, 717, 515, 285, 801, 693, 174, 562, 596, 116, 655,
       200,  83, 837, 211, 328, 622,  48, 249,  88, 291,  90, 556, 427,
       318, 790, 673, 370, 127, 677, 857, 263, 101,  30, 875, 605, 283,
       479, 916, 804,  25, 594, 900,  13, 635,  14,   3, 599, 197, 571,
        70,  51,  76, 907,  24, 404, 354, 159, 541,  91, 375, 922, 674,
       225, 752, 180, 759, 227, 855, 331, 739, 355, 772, 905, 324, 871,
       574, 665, 561, 299, 336, 438, 499, 453, 554,  49, 383])

2016-02-01 2016-02-08 2016-02-15 2016-02-22	2016-02-29	2016-03-07 2016-03-14 2016-03-21 2016-03-28 2016-04-04 2016-04-11 2016-04-15

In [ ]:
from sklearn.metrics import recall_score 
from sklearn.metrics import accuracy_score

def select_label(df, start_time, end_time, product):
    df = df[(df.time >= start_time) & (df.time < end_time) &(df['type'] == 4)]
    df = pd.merge(df, product, how='inner', on='sku_id')
    return df

def select_range(df, start_time, end_time):
    df = df[(df.time >= start_time) & (df.time < end_time)]
    return df


def report(pred, label):
    
    actions = label
    result = pred
    
        # 所有用户商品对
    all_user_item_pair = actions['user_id'].map(str) + '-' + actions['sku_id'].map(str)
    all_user_item_pair = np.array(all_user_item_pair)
    # 所有购买用户
    all_user_set = actions['user_id'].unique()
    
    
    # 所有品类中预测购买的用户
    all_user_test_set = result['user_id'].unique()
    all_user_test_item_pair = result['user_id'].map(str) + '-' + result['sku_id'].map(str)
    all_user_test_item_pair = np.array(all_user_test_item_pair)
    

    #计算所有用户购买评价指标
    pos, neg = 0,0
    for user_id in all_user_test_set:
        if user_id in all_user_set:
            pos += 1
        else:
            neg += 1
    all_user_acc = 1.0 * pos / ( pos + neg)
    all_user_recall = 1.0 * pos / len(all_user_set)
    print '所有用户中预测购买用户的准确率为 ' + str(all_user_acc)
    print '所有用户中预测购买用户的召回率' + str(all_user_recall)
    
    pos, neg = 0, 0
    for user_item_pair in all_user_test_item_pair:
        if user_item_pair in all_user_item_pair:
            pos += 1
        else:
            neg += 1
    all_item_acc = 1.0 * pos / ( pos + neg)
    all_item_recall = 1.0 * pos / len(all_user_item_pair)
    print '所有用户中预测购买商品的准确率为 ' + str(all_item_acc)
    print '所有用户中预测购买商品的召回率' + str(all_item_recall)
    F11 = 6.0 * all_user_recall * all_user_acc / (5.0 * all_user_recall + all_user_acc)
    F12 = 5.0 * all_item_acc * all_item_recall / (2.0 * all_item_recall + 3 * all_item_acc)
    score = 0.4 * F11 + 0.6 * F12
    print 'F11=' + str(F11)
    print 'F12=' + str(F12)
    print 'score=' + str(score)



    

In [ ]:
start_time = ["2016-02-01", "2016-02-08", "2016-02-15", "2016-02-22", "2016-02-29", "2016-03-07", "2016-03-14", "2016-03-21", "2016-03-28", 
              "2016-04-04", "2016-04-11", "2016-04-15"]

# 下单
# all_test_actions = action_1[(action_1.time >= start_time[2]) & (action_1.time < start_time[3]) & ((action_1['type'] == 2) | (action_1['type'] == 5))]

all_test_actions = action_1[(action_1.time >= start_time[2]) & (action_1.time < start_time[3]) & (action_1['type'] == 2)]
# 购买
all_real_actions = action_1[(action_1.time >= start_time[3]) & (action_1.time < start_time[4]) & (action_1['type'] == 4)]

report(all_test_actions, all_real_actions)

test_actions = all_test_actions.groupby(['user_id', 'sku_id'])['time'].agg({'no':'count'}).reset_index().sort_values(by='no',ascending=False).groupby('user_id').first()

test_actions = test_actions.reset_index()

# 下单
test_actions = pd.merge(test_actions, product, how='inner', on='sku_id')

# 购买
real_actions = pd.merge(all_real_actions, product, how='inner', on='sku_id')

report(test_actions, real_actions)


In [ ]:
action_reg = action_3[(action_3.time >= '2016-04-13') & (action_3.time < '2016-04-16') & (action_3['type'] == 2)]
A = action_reg.groupby(['user_id', 'sku_id'])['time'].agg({'no':'count'}).reset_index().sort_values(by='no',ascending=False).groupby('user_id').first()
A_clean = A.reset_index()[['user_id', 'sku_id']]
B = product
A_B = pd.merge(A_clean, B, how='inner', on='sku_id')
A_B[['user_id', 'sku_id']].to_csv('submision.csv', index=False)

#action_reg.groupby(['user_id', 'sku_id']).count().reset_index()[['user_id', 'sku_id']].to_csv('submisson.csv', index=False)

In [ ]:
print 'hello'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from datetime import datetime
import math

def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

def weighted(cols):
    cols['action_1'] = sum(cols['action_1'] * cols['weights'])
    cols['action_2'] = sum(cols['action_2'] * cols['weights'])
    cols['action_3'] = sum(cols['action_3'] * cols['weights'])
    cols['action_4'] = sum(cols['action_4'] * cols['weights'])
    cols['action_5'] = sum(cols['action_5'] * cols['weights'])
    cols['action_6'] = sum(cols['action_6'] * cols['weights'])
    return cols
    
    
def make_train_set(train_start_date, train_end_date, test_end_date, actions, user, comment):
    
    comment_date = ["2016-02-01", "2016-02-08", "2016-02-15", "2016-02-22", "2016-02-29", "2016-03-07", "2016-03-14", "2016-03-21", "2016-03-28", 
              "2016-04-04", "2016-04-11", "2016-04-15"]
    
    comment_date_end = train_end_date
    comment_date_begin = comment_date[0]
    
    for date in reversed(comment_date):
        if date < comment_date_end:
            comment_date_begin = date
        
    
    buy_action = actions[(actions.time >= train_end_date) & (actions.time < test_end_date) & (actions['type'] == 4)]
    buy_action = buy_action.groupby(['user_id', 'sku_id'], as_index=False).sum()
    buy_action['label'] = 1 
    buy_action = buy_action[['user_id', 'sku_id', 'label']]
    
    
    comment = comment[(comment.dt >= comment_date_begin) & (comment.dt < comment_date_end)]
    
    
    actions = actions[(actions.time >= train_start_date) & (actions.time < train_end_date)]
    #comment = comment[]
    
    df = pd.get_dummies(actions['type'],prefix='action')
    actions = pd.concat([actions, df], axis=1)
    
    # 处理时间
    actions['datetime'] = actions['time'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    actions['weights'] =  datetime.strptime(train_end_date, '%Y-%m-%d') - actions['datetime']
    
    #to days
    actions['weights'] = actions['weights'].map(lambda x: math.exp(-x.days))
    
    actions['action_1'] = actions['action_1'] * actions['weights']
    actions['action_2'] = actions['action_2'] * actions['weights']
    actions['action_3'] = actions['action_3'] * actions['weights']
    actions['action_4'] = actions['action_4'] * actions['weights']
    actions['action_5'] = actions['action_5'] * actions['weights']
    actions['action_6'] = actions['action_6'] * actions['weights']
    
    
    #drop model id
    del actions['model_id']
    del actions['type']
    del actions['time']
    del actions['datetime']
    del actions['weights']
    
    actions = actions.groupby(['user_id', 'sku_id', 'cate', 'brand'], as_index=False).sum()
    actions = pd.merge(actions, user, how='left', on='user_id')
    actions = pd.merge(actions, buy_action, how='left', on=['user_id', 'sku_id'])
    actions = pd.merge(actions, comment, how='left', on='sku_id')
    actions = actions.fillna(0)
    
    #actions = pd.merge(actions, comment, how='left', on='user_id')
    # process time
    
    #print actions
    #group by 
    #group_actions = actions.groupby(['user_id', 'sku_id', 'cate', 'brand'])
    #actions = group_actions.apply(weighted)
    return actions
    #sss = group_actions.apply(lambda x : pd.Series([x['action_1'].sum()], index=['sss']))
    #print sss
    

start_date = "2016-02-01"
end_date = "2016-02-08"
test_end_date = "2016-02-12"
#print action_1
actions = make_train_set(start_date, end_date, test_end_date, action_1, user, comment)




In [ ]:
#buy_action = action_1[(action_1.time >= end_date) & (action_1.time < test_end_date) & (action_1['type'] == 4)]
#buy_action = buy_action.groupby(['user_id', 'sku_id'], as_index=False).sum()
#len(buy_action.index)
#len(actions[actions.label == 1].index)

In [ ]:
buy_action

In [ ]:
start_date = "2016-02-08"
end_date = "2016-02-15"
test_end_date = "2016-02-19"
#print action_1
actions = make_train_set(start_date, end_date, test_end_date, action_1, user, comment)


train_data = actions[numerical_columns]

for col in categorical_columns:
    df = pd.get_dummies(actions[col],prefix=col)
    train_data = pd.concat([train_data, df], axis=1)

label_data = actions[label_columns]

dpred = xgb.DMatrix(train_data)
ypred_bst = bst.predict(dpred)
actions['predict'] = ypred_bst

threadhold = 0.3

all_test_actions = actions[actions.predict > threadhold]
#len(all_test_actions.index)
test_begin_date = "2016-02-15"
test_end_date = "2016-02-19"

all_real_actions = action_1[(action_1.time >= test_begin_date) & (action_1.time < test_end_date) & (action_1['type'] == 4)]

report(all_test_actions, all_real_actions)

#test_actions = all_test_actions.groupby(['user_id', 'sku_id'])['time'].agg({'no':'count'}).reset_index().sort_values(by='no',ascending=False).groupby('user_id').first()

#test_actions = test_actions.reset_index()

# 下单
test_actions = pd.merge(all_test_actions, product, how='inner', on='sku_id')

# 购买
real_actions = pd.merge(all_real_actions, product, how='inner', on='sku_id')

report(test_actions, real_actions)


In [ ]:
%matplotlib inline
from sklearn import metrics

dpred = xgb.DMatrix(train_data)
ypred_bst = bst.predict(dpred)


fpr, tpr, threshold = metrics.roc_curve(label_data, ypred_bst)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
#from sklearn.metrics import roc_auc_score
#roc_auc_score(label_data, ypred_bst)

In [ ]:
action_reg = action_3[(action_3.time >= '2016-04-13') & (action_3.time < '2016-04-16') & (action_3['type'] == 2)]
A = action_reg.groupby(['user_id', 'sku_id'])['time'].agg({'no':'count'}).reset_index().sort_values(by='no',ascending=False).groupby('user_id').first()
A = action_reg.groupby(['user_id', 'sku_id'])['time'].agg({'no':'count'})
A_clean = A.reset_index()[['user_id', 'sku_id']]
B = product
A_clean[['user_id', 'sku_id']].to_csv('submision.csv', index=False)

#A_clean[['user_id', 'sku_id']].to_csv('submision.csv', index=False)
#A.head()
A_B = pd.merge(A_clean, B, how='inner', on='sku_id')
#A_B
len(A_B.index)
#A_B[['user_id', 'sku_id']].to_csv('submision.csv', index=False)

In [ ]:
#action_reg.groupby(['user_id', 'sku_id']).count().reset_index()[['user_id', 'sku_id']].to_csv('submisson.csv', index=False)